# Delta Lake Rollback with RESTORE

In [4]:
df = spark.range(0, 3)

In [5]:
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
+---+



In [6]:
df.write.format("delta").save("tmp/delta-table")

In [7]:
df2 = spark.range(4, 6)

In [8]:
df2.write.mode("overwrite").format("delta").save("tmp/delta-table")

In [9]:
df2.show()

+---+
| id|
+---+
|  4|
|  5|
+---+



In [10]:
df3 = spark.range(7, 10)

In [11]:
df3.show()

+---+
| id|
+---+
|  7|
|  8|
|  9|
+---+



In [12]:
df3.write.mode("overwrite").format("delta").save("tmp/delta-table")

In [13]:
spark.read.format("delta").load("tmp/delta-table").show()

+---+
| id|
+---+
|  9|
|  7|
|  8|
+---+



In [4]:
df = spark.range(0, 3)

In [5]:
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
+---+



In [6]:
df.write.format("delta").save("tmp/delta-table")

In [7]:
df2 = spark.range(4, 6)

In [8]:
df2.write.mode("overwrite").format("delta").save("tmp/delta-table")

In [9]:
df2.show()

+---+
| id|
+---+
|  4|
|  5|
+---+



In [10]:
df3 = spark.range(7, 10)

In [11]:
df3.show()

+---+
| id|
+---+
|  7|
|  8|
|  9|
+---+



In [12]:
df3.write.mode("overwrite").format("delta").save("tmp/delta-table")

In [13]:
spark.read.format("delta").load("tmp/delta-table").show()

+---+
| id|
+---+
|  9|
|  7|
|  8|
+---+



In [1]:
import pyspark
from delta import *

In [2]:
builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
    .config("spark.databricks.delta.retentionDurationCheck.enabled", "false")
)

In [3]:
spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/matthew.powers/opt/miniconda3/envs/pyspark-322-delta-200/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/matthew.powers/.ivy2/cache
The jars for the packages stored in: /Users/matthew.powers/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d0cb9596-04e6-4ce0-85cc-62548df9322b;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.0.0 in central
	found io.delta#delta-storage;2.0.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 388ms :: artifacts dl 26ms
	:: modules in use:
	io.delta#delta-core_2.12;2.0.0 from central in [default]
	io.delta#delta-storage;2.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number|

## Create Delta Lake table

In [4]:
df = spark.range(0, 3)

In [5]:
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
+---+



In [6]:
df.write.format("delta").save("tmp/delta-table")

In [7]:
df2 = spark.range(4, 6)

In [8]:
df2.write.mode("overwrite").format("delta").save("tmp/delta-table")

In [9]:
df2.show()

+---+
| id|
+---+
|  4|
|  5|
+---+



In [10]:
df3 = spark.range(7, 10)

In [11]:
df3.show()

+---+
| id|
+---+
|  7|
|  8|
|  9|
+---+



In [12]:
df3.write.mode("overwrite").format("delta").save("tmp/delta-table")

In [13]:
spark.read.format("delta").load("tmp/delta-table").show()

+---+
| id|
+---+
|  9|
|  7|
|  8|
+---+



In [14]:
!tree tmp/delta-table

tmp/delta-table
├── _delta_log
│   ├── 00000000000000000000.json
│   ├── 00000000000000000001.json
│   └── 00000000000000000002.json
├── part-00000-2f9e54f7-e443-4fb8-b3f3-6e265ef4a5d6-c000.snappy.parquet
├── part-00000-64f58004-ce02-41cd-8cef-d46a825e086f-c000.snappy.parquet
├── part-00000-8b2a03e6-01ca-4570-91f5-d164e1ea2578-c000.snappy.parquet
├── part-00003-89e8fc18-b469-44ad-ac70-9bb00ca7c4ac-c000.snappy.parquet
├── part-00003-ddc71a88-701d-4883-b7d0-dcba8a65c9f7-c000.snappy.parquet
├── part-00004-806caf5e-02e1-4e01-954b-1ae45500db30-c000.snappy.parquet
├── part-00006-0d450d2c-822b-454f-9d9f-031bfc056bfa-c000.snappy.parquet
├── part-00006-ba99b2c0-188d-4a74-9779-d88350f63111-c000.snappy.parquet
├── part-00009-45c9e2c0-cd34-4c9f-b3cd-1f39e5ceb369-c000.snappy.parquet
├── part-00009-757b7f2a-f967-4302-a996-0d2b2999f817-c000.snappy.parquet
└── part-00009-bdb964bc-8345-4d57-91e0-6190a6d1132e-c000.snappy.parquet

1 directory, 14 files


## Time travel to different versions of Delta Lake

In [13]:
spark.read.format("delta").option("versionAsOf", "0").load("tmp/delta-table").show()

+---+
| id|
+---+
|  0|
|  2|
|  1|
+---+



In [14]:
spark.read.format("delta").option("versionAsOf", "1").load("tmp/delta-table").show()

+---+
| id|
+---+
|  4|
|  5|
+---+



In [15]:
spark.read.format("delta").option("versionAsOf", "2").load("tmp/delta-table").show()

+---+
| id|
+---+
|  7|
|  9|
|  8|
+---+



## Restore v1 of Delta Lake

In [17]:
from delta.tables import *

In [18]:
deltaTable = DeltaTable.forPath(spark, "tmp/delta-table")

In [19]:
deltaTable.restoreToVersion(1)

DataFrame[table_size_after_restore: bigint, num_of_files_after_restore: bigint, num_removed_files: bigint, num_restored_files: bigint, removed_files_size: bigint, restored_files_size: bigint]

In [20]:
spark.read.format("delta").load("tmp/delta-table").show()

+---+
| id|
+---+
|  4|
|  5|
+---+



In [21]:
spark.read.format("delta").option("versionAsOf", "2").load("tmp/delta-table").show()

+---+
| id|
+---+
|  9|
|  7|
|  8|
+---+



In [22]:
!tree tmp/delta-table

tmp/delta-table
├── _delta_log
│   ├── 00000000000000000000.json
│   ├── 00000000000000000001.json
│   ├── 00000000000000000002.json
│   ├── 00000000000000000003.checkpoint.parquet
│   ├── 00000000000000000003.json
│   └── _last_checkpoint
├── part-00000-2f9e54f7-e443-4fb8-b3f3-6e265ef4a5d6-c000.snappy.parquet
├── part-00000-64f58004-ce02-41cd-8cef-d46a825e086f-c000.snappy.parquet
├── part-00000-8b2a03e6-01ca-4570-91f5-d164e1ea2578-c000.snappy.parquet
├── part-00003-89e8fc18-b469-44ad-ac70-9bb00ca7c4ac-c000.snappy.parquet
├── part-00003-ddc71a88-701d-4883-b7d0-dcba8a65c9f7-c000.snappy.parquet
├── part-00004-806caf5e-02e1-4e01-954b-1ae45500db30-c000.snappy.parquet
├── part-00006-0d450d2c-822b-454f-9d9f-031bfc056bfa-c000.snappy.parquet
├── part-00006-ba99b2c0-188d-4a74-9779-d88350f63111-c000.snappy.parquet
├── part-00009-45c9e2c0-cd34-4c9f-b3cd-1f39e5ceb369-c000.snappy.parquet
├── part-00009-757b7f2a-f967-4302-a996-0d2b2999f817-c000.snappy.parquet
└── part-00009-bdb964bc-8345-4d57-91e0-6

In [49]:
!cat tmp/delta-table/_delta_log/00000000000000000000.json

{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}
{"metaData":{"id":"1a722b06-05b2-41a5-a2a5-91d21c51c94a","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"id\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1664563482176}}
{"add":{"path":"part-00000-64f58004-ce02-41cd-8cef-d46a825e086f-c000.snappy.parquet","partitionValues":{},"size":296,"modificationTime":1664563484686,"dataChange":true,"stats":"{\"numRecords\":0,\"minValues\":{},\"maxValues\":{},\"nullCount\":{}}"}}
{"add":{"path":"part-00003-89e8fc18-b469-44ad-ac70-9bb00ca7c4ac-c000.snappy.parquet","partitionValues":{},"size":478,"modificationTime":1664563484721,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"id\":0},\"maxValues\":{\"id\":0},\"nullCount\":{\"id\":0}}"}}
{"add":{"path":"part-00006-ba99b2c0-188d-4a74-9779-d88350f63111-c000.snappy.parquet","partitionValues":{},"size":478,"modifi

In [50]:
!cat tmp/delta-table/_delta_log/00000000000000000001.json

{"add":{"path":"part-00000-8b2a03e6-01ca-4570-91f5-d164e1ea2578-c000.snappy.parquet","partitionValues":{},"size":296,"modificationTime":1664563499429,"dataChange":true,"stats":"{\"numRecords\":0,\"minValues\":{},\"maxValues\":{},\"nullCount\":{}}"}}
{"add":{"path":"part-00004-806caf5e-02e1-4e01-954b-1ae45500db30-c000.snappy.parquet","partitionValues":{},"size":478,"modificationTime":1664563499429,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"id\":4},\"maxValues\":{\"id\":4},\"nullCount\":{\"id\":0}}"}}
{"add":{"path":"part-00009-757b7f2a-f967-4302-a996-0d2b2999f817-c000.snappy.parquet","partitionValues":{},"size":478,"modificationTime":1664563499429,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"id\":5},\"maxValues\":{\"id\":5},\"nullCount\":{\"id\":0}}"}}
{"remove":{"path":"part-00009-45c9e2c0-cd34-4c9f-b3cd-1f39e5ceb369-c000.snappy.parquet","deletionTimestamp":1664563500711,"dataChange":true,"extendedFileMetadata":true,"partitionValues":{},"size":478}}


In [23]:
!cat tmp/delta-table/_delta_log/00000000000000000002.json

{"add":{"path":"part-00000-2f9e54f7-e443-4fb8-b3f3-6e265ef4a5d6-c000.snappy.parquet","partitionValues":{},"size":296,"modificationTime":1664563516040,"dataChange":true,"stats":"{\"numRecords\":0,\"minValues\":{},\"maxValues\":{},\"nullCount\":{}}"}}
{"add":{"path":"part-00003-ddc71a88-701d-4883-b7d0-dcba8a65c9f7-c000.snappy.parquet","partitionValues":{},"size":478,"modificationTime":1664563516059,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"id\":7},\"maxValues\":{\"id\":7},\"nullCount\":{\"id\":0}}"}}
{"add":{"path":"part-00006-0d450d2c-822b-454f-9d9f-031bfc056bfa-c000.snappy.parquet","partitionValues":{},"size":478,"modificationTime":1664563516047,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"id\":8},\"maxValues\":{\"id\":8},\"nullCount\":{\"id\":0}}"}}
{"add":{"path":"part-00009-bdb964bc-8345-4d57-91e0-6190a6d1132e-c000.snappy.parquet","partitionValues":{},"size":478,"modificationTime":1664563516047,"dataChange":true,"stats":"{\"numRecords\":1,\"minVa

In [24]:
!cat tmp/delta-table/_delta_log/00000000000000000003.json

{"commitInfo":{"timestamp":1664563576249,"operation":"RESTORE","operationParameters":{"version":1,"timestamp":null},"readVersion":2,"isolationLevel":"Serializable","isBlindAppend":false,"operationMetrics":{"numRestoredFiles":"3","removedFilesSize":"1730","numRemovedFiles":"4","restoredFilesSize":"1252","numOfFilesAfterRestore":"3","tableSizeAfterRestore":"1252"},"engineInfo":"Apache-Spark/3.2.2 Delta-Lake/2.0.0","txnId":"7bbec476-1d7c-49a7-80f2-d9c26ed4ba58"}}
{"metaData":{"id":"1a722b06-05b2-41a5-a2a5-91d21c51c94a","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"id\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1664563482176}}
{"add":{"path":"part-00000-8b2a03e6-01ca-4570-91f5-d164e1ea2578-c000.snappy.parquet","partitionValues":{},"size":296,"modificationTime":1664563499429,"dataChange":true,"stats":"{\"numRecords\":0,\"minValues\":{},\"maxValues\":{},\"nullCount

In [25]:
spark.read.format("delta").option("versionAsOf", "2").load("tmp/delta-table").show()

+---+
| id|
+---+
|  9|
|  7|
|  8|
+---+



In [27]:
spark.read.format("delta").option("versionAsOf", "1").load("tmp/delta-table").show()

+---+
| id|
+---+
|  4|
|  5|
+---+



In [26]:
spark.read.format("delta").load("tmp/delta-table").show()

+---+
| id|
+---+
|  4|
|  5|
+---+



## Vacuum Delta Lake

In [28]:
# Two different ways to vacuum
deltaTable.vacuum(retentionHours=0)
# spark.sql("VACUUM 'tmp/delta-table'")

Deleted 8 files and directories in a total of 1 directories.


DataFrame[]

In [29]:
spark.read.format("delta").load("tmp/delta-table").show()

+---+
| id|
+---+
|  4|
|  5|
+---+



In [30]:
spark.read.format("delta").option("versionAsOf", "2").load("tmp/delta-table").show()

22/09/30 14:51:20 ERROR Executor: Exception in task 0.0 in stage 180.0 (TID 12139)
java.io.FileNotFoundException: 
File file:/Users/matthew.powers/Documents/code/my_apps/delta-examples/notebooks/pyspark/tmp/delta-table/part-00009-bdb964bc-8345-4d57-91e0-6190a6d1132e-c000.snappy.parquet does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:506)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:130)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:187)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:104)


Py4JJavaError: An error occurred while calling o99.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 180.0 failed 1 times, most recent failure: Lost task 0.0 in stage 180.0 (TID 12139) (qtk9h72yp0 executor driver): java.io.FileNotFoundException: 
File file:/Users/matthew.powers/Documents/code/my_apps/delta-examples/notebooks/pyspark/tmp/delta-table/part-00009-bdb964bc-8345-4d57-91e0-6190a6d1132e-c000.snappy.parquet does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:506)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:130)
	at org.apache.spark.s

In [32]:
spark.read.format("delta").option("versionAsOf", "1").load("tmp/delta-table").show()

+---+
| id|
+---+
|  4|
|  5|
+---+



In [33]:
spark.read.format("delta").option("versionAsOf", "0").load("tmp/delta-table").show()

22/09/30 14:52:39 ERROR Executor: Exception in task 0.0 in stage 209.0 (TID 12454)
java.io.FileNotFoundException: 
File file:/Users/matthew.powers/Documents/code/my_apps/delta-examples/notebooks/pyspark/tmp/delta-table/part-00009-45c9e2c0-cd34-4c9f-b3cd-1f39e5ceb369-c000.snappy.parquet does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:506)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:130)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:187)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:104)


Py4JJavaError: An error occurred while calling o114.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 209.0 failed 1 times, most recent failure: Lost task 0.0 in stage 209.0 (TID 12454) (qtk9h72yp0 executor driver): java.io.FileNotFoundException: 
File file:/Users/matthew.powers/Documents/code/my_apps/delta-examples/notebooks/pyspark/tmp/delta-table/part-00009-45c9e2c0-cd34-4c9f-b3cd-1f39e5ceb369-c000.snappy.parquet does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:506)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:130)
	at org.apache.spark.

In [34]:
spark.read.format("delta").option("versionAsOf", "3").load("tmp/delta-table").show()

+---+
| id|
+---+
|  4|
|  5|
+---+



In [36]:
spark.sql("DESCRIBE HISTORY default.`tmp/delta-table`")

AnalysisException: Table or view 'tmp/delta-table' not found in database 'default'

## Cannot time travel back after the vacuum

In [24]:
spark.read.format("delta").option("versionAsOf", "0").load("tmp/delta-table").show()

22/08/28 19:11:50 ERROR Executor: Exception in task 0.0 in stage 188.0 (TID 12231)
java.io.FileNotFoundException: 
File file:/Users/matthew.powers/Documents/code/my_apps/delta-examples/notebooks/pyspark/tmp/delta-table/part-00003-367d5d44-9e72-4e94-822e-891f05001843-c000.snappy.parquet does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:506)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:130)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:187)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:104)


Py4JJavaError: An error occurred while calling o107.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 188.0 failed 1 times, most recent failure: Lost task 0.0 in stage 188.0 (TID 12231) (qtk9h72yp0 executor driver): java.io.FileNotFoundException: 
File file:/Users/matthew.powers/Documents/code/my_apps/delta-examples/notebooks/pyspark/tmp/delta-table/part-00003-367d5d44-9e72-4e94-822e-891f05001843-c000.snappy.parquet does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:506)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:130)
	at org.apache.spark.

## Restore SQL Syntax

In [37]:
df = spark.range(0, 3)

In [38]:
df.write.format("delta").saveAsTable("my_amazing_table")

In [39]:
df2 = spark.range(4, 6)

In [40]:
df2.write.mode("overwrite").format("delta").saveAsTable("my_amazing_table")

In [41]:
df3 = spark.range(7, 10)

In [42]:
df3.write.mode("overwrite").format("delta").saveAsTable("my_amazing_table")

In [43]:
spark.sql("SELECT * FROM my_amazing_table").show()

+---+
| id|
+---+
|  7|
|  8|
|  9|
+---+



In [47]:
spark.sql("RESTORE TABLE my_amazing_table TO VERSION AS OF 1")

DataFrame[table_size_after_restore: bigint, num_of_files_after_restore: bigint, num_removed_files: bigint, num_restored_files: bigint, removed_files_size: bigint, restored_files_size: bigint]

In [48]:
spark.sql("SELECT * FROM my_amazing_table").show()

+---+
| id|
+---+
|  4|
|  5|
+---+



## Cleanup

In [70]:
%rm -rf tmp